In [1]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD

/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhaoliang/anaconda3/lib/pyth

In [2]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from collections import Counter

In [3]:
train = pd.read_csv('../data/train.csv')
label = pd.read_csv('../data/train_label.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/submission.csv')
train_data = pd.merge(train,label,on='ID')

/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def deal_time(x):
    if str(x) == 'nan':
        return x
    
    x = str(x)
    parts = x.split(':')
    return int(parts[0]) + float(parts[1])/100

def cal_diff(x, y):
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    else:
        return y-x
    
def deal_manage_range(x):
    g = json.loads(x)
    return len(g)

def deal_cat(x):
    if str(x) == 'nan':
        return str(x)
    return str(x)

def get_built_tax_rate(x, y, z):  #x:cj y:zz z:xf
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    elif str(z) == 'nan':
        return z
    elif y+z==0:
        return 0
    else:
        return x/(y+z)
    
def get_xf_tax(x, y): #消费税 x:jy  y:zz
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    return x/0.03 - y

def get_tax(x, y): #y->tax
    if str(x) == 'nan':
        return x
    elif str(y) == 'nan':
        return y
    elif y == 0:
        return 0
    return x/y

all_ranges = []
jyfw = list(train_data['经营范围'])
for item in jyfw:
    all_ranges += json.loads(item)
range_dict = dict(Counter(all_ranges))
# bins = [4,8,16,64,128,256,600,3000]
def deal_manage_range_bin(x, lower, upper=100000):
    count = 0
    items = json.loads(x)
    for item in items:
        counts = range_dict.get(item)
        if counts and counts<=upper and counts>lower:
            count += 1
    return count

In [5]:
train_data['经营期限至'] = train_data['经营期限至'].apply(deal_time)
train_data['经营期限自'] = train_data['经营期限自'].apply(deal_time)
train_data['成立日期'] = train_data['成立日期'].apply(deal_time)
train_data['核准日期'] = train_data['核准日期'].apply(deal_time)
train_data['注销时间'] = train_data['注销时间'].apply(deal_time)

train_data['消费税'] = train_data.apply(lambda row: get_xf_tax(row['教育费'], row['增值税']), axis=1)
train_data['城建税率'] = train_data.apply(lambda row: get_built_tax_rate(row['城建税'], row['增值税'], row['消费税']), axis=1)
train_data['应纳税额'] = train_data.apply(lambda row: get_tax(row['企业所得税'], row['城建税率']), axis=1)

train_data['经营范围1'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(0,4,))
train_data['经营范围2'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(4,8,))
train_data['经营范围3'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(8,16,))
train_data['经营范围4'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(16,64,))
train_data['经营范围5'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(64,128,))
train_data['经营范围6'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(128,256,))
train_data['经营范围7'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(256,600,))
train_data['经营范围8'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(600,3000,))
train_data['经营范围9'] = train_data['经营范围'].apply(deal_manage_range_bin, args=(3000,))

train_data['经营范围'] = train_data['经营范围'].apply(deal_manage_range)
for item in train_data.columns:
    parts = item.split("_")
    if len(parts) == 2:
        if parts[1] == '年初数':
            train_data[parts[0] + '_diff'] = train_data.apply(lambda row: cal_diff(row[item], row[parts[0]+"_年末数"]), axis=1)

In [6]:
train_data = train_data.fillna(-1)
drop_feat = ['ID','邮政编码','Label','管辖机关', '企业类别','行业门类','行业代码','企业状态','邮政编码','登记机关','企业类型']
feat = [item for item in train_data.columns if item not in drop_feat]
# cat_feat = ['管辖机关', '企业类别','行业门类','行业代码','企业状态','邮政编码','登记机关','企业类型']
X = train_data[feat]
y = train_data['Label'].values
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [7]:
from sklearn.metrics import roc_auc_score
import keras
class roc_callback(keras.callbacks.Callback):
    def __init__(self,training_data, validation_data):
        
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
    
    def on_train_begin(self, logs={}):
        return
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):        
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)      
        
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)      
        
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        return   



In [8]:
from keras import backend as K
import tensorflow as tf
'''
Compatible with tensorflow backend
'''
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(K.epsilon()+pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
    return focal_loss_fixed


In [14]:
from keras.optimizers import  adam
ad = adam(0.1)

In [44]:
input_layer = Input(shape=(258,))
dense1 = Dense(100)(input_layer)
dense2 = Dense(200)(dense1)
dense2 = Dense(50)(dense2)
dense2 = Dense(300)(dense2)
dense2 = Dense(50)(dense2)
dense2 = Dense(50)(dense2)
dense3 = Dense(25)(dense2)
output = Dense(1, activation='sigmoid')(dense3)

model = Model(input_layer, output)
model.compile(optimizer='rmsprop',  # lr = 1e-3
              loss=focal_loss(),
              metrics=['accuracy'])

In [45]:
cw = {0:10, 1:90}
model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks = [roc_callback(training_data=[x_train, y_train], validation_data=[x_test, y_test])],epochs=200000,class_weight=cw)

Train on 10056 samples, validate on 4310 samples
Epoch 1/200000
10056/10056 [==============================] - 3s 342us/step - loss: 233.5749 - acc: 0.9168 - val_loss: 8.3159 - val_acc: 0.9390
roc-auc: 0.499 - roc-auc_val: 0.5026                                                                                                    
Epoch 2/200000
10056/10056 [==============================] - 2s 153us/step - loss: 209.0309 - acc: 0.9222 - val_loss: 8.6069 - val_acc: 0.9383
roc-auc: 0.4977 - roc-auc_val: 0.4986                                                                                                    
Epoch 3/200000
10056/10056 [==============================] - 2s 160us/step - loss: 186.5205 - acc: 0.9267 - val_loss: 8.1581 - val_acc: 0.9394
roc-auc: 0.4996 - roc-auc_val: 0.4991                                                                                                    
Epoch 4/200000
10056/10056 [==============================] - 2s 155us/step - loss: 187.6614 - acc: 0.9267

10056/10056 [==============================] - 1s 108us/step - loss: 162.3350 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 31/200000
10056/10056 [==============================] - 1s 108us/step - loss: 164.3225 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 32/200000
10056/10056 [==============================] - 1s 107us/step - loss: 165.6368 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 33/200000
10056/10056 [==============================] - 1s 108us/step - loss: 165.1239 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-

10056/10056 [==============================] - 1s 83us/step - loss: 166.1177 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 60/200000
10056/10056 [==============================] - 1s 82us/step - loss: 164.3866 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 61/200000
10056/10056 [==============================] - 1s 82us/step - loss: 164.9957 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 62/200000
10056/10056 [==============================] - 1s 83us/step - loss: 163.0402 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_

10056/10056 [==============================] - 2s 181us/step - loss: 164.1622 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 89/200000
10056/10056 [==============================] - 2s 182us/step - loss: 163.9699 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 90/200000
10056/10056 [==============================] - 2s 188us/step - loss: 166.5024 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 91/200000
10056/10056 [==============================] - 2s 185us/step - loss: 166.4703 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-

10056/10056 [==============================] - 2s 169us/step - loss: 165.5407 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 118/200000
10056/10056 [==============================] - 2s 166us/step - loss: 164.1302 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 119/200000
10056/10056 [==============================] - 2s 164us/step - loss: 167.2076 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 120/200000
10056/10056 [==============================] - 2s 161us/step - loss: 163.6173 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - r

10056/10056 [==============================] - 1s 140us/step - loss: 164.8034 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 147/200000
10056/10056 [==============================] - 1s 129us/step - loss: 165.6689 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 148/200000
10056/10056 [==============================] - 2s 149us/step - loss: 163.1364 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 149/200000
10056/10056 [==============================] - 1s 146us/step - loss: 164.1622 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - r

10056/10056 [==============================] - 1s 92us/step - loss: 167.0794 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 176/200000
10056/10056 [==============================] - 1s 93us/step - loss: 164.1302 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 177/200000
10056/10056 [==============================] - 1s 91us/step - loss: 163.1044 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-auc_val: 0.4995                                                                                                    
Epoch 178/200000
10056/10056 [==============================] - 1s 90us/step - loss: 163.1685 - acc: 0.9304 - val_loss: 7.9786 - val_acc: 0.9399
roc-auc: 0.4992 - roc-a

KeyboardInterrupt: 